<a href="https://colab.research.google.com/github/kmouleejunhyuk/NLP_KOBERT-review/blob/main/nh_origin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#트라이할 것: no duplicate treatment, ord
#max: n=3, dr 0.5, lr 5e-5, 중복제거, content_title_ord
#현샅태: 
status='html_128'

In [ ]:
#######라이브러리 버젼#######
#mxnet-cu101: 1.7.0
#gluonnlp: 0.10.0
#pandas: 1.1.4
#tqdm: 4.53.0??
#sentencepiece: 0.1.85
#transformers: 2.1.1
#torch: 1.3.1
#numpy: 1.18.5
#kobert: 0.1.1

#######개발 환경#######
#google colab, python
#런타임 가속기 유형: GPU, Tesla T4 or p100 사용, Tesla P4나 k80에서는 메모리 문제로 OOM 에러 발생

In [ ]:
!nvidia-smi

Sun Dec 20 12:05:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    12W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#구글 드라이브 마운트 & 경로설정
from google.colab import drive
drive.mount('/gdrive')

PATH='/gdrive/My Drive/TRAIN'

Mounted at /gdrive


In [ ]:
#필요 라이브러리 설치
!pip install mxnet-cu101 &> /dev/null
!pip install gluonnlp pandas tqdm &> /dev/null
!pip install sentencepiece==0.1.85 &> /dev/null
!pip install transformers==2.1.1 &> /dev/null
!pip install torch==1.3.1 &> /dev/null

#kobert 설치
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master &> /dev/null

In [ ]:
#라이브러리 로드
import pandas as pd
import numpy as np
from os import path
from shutil import copy

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import random
import time

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import WarmupLinearSchedule

import html

In [ ]:
#seed 
random_seed=44159   #44159
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
#train, test dataset 로드
raw_train= pd.read_table(path.join(PATH, 'news_train.csv'), sep=',').astype('string')
raw_test=pd.read_table(path.join(PATH, 'news_test.csv'), sep=',').astype('string')
#print(raw_train.columns)
#print(raw_test.columns)

In [ ]:
import re
q='2020년 한국 TV 2대중 1대 인터넷 연결된다._//_ 아미코젠, 52주 신고가_//_ 3 '

def replacer(input: str, label_dic: dict):
    for key, value in label_dic.items():
        input=input.replace(key, value)
    return input

def multi_re(input: str):
    #input=replacer(input, {'獨': '독일 ', '伊': '이탈리아 ', '佛': '프랑스 ', 
    #                       '日': '일본 ', '韓': '한국 ','中': '중국 ' ,'美': '미국 ',
    #                       '北': '북한 ','露': '러시아', '英':'영국', '印': '인도',
    #                       '女':'여', '男':'남', 
    #                       '前':'전 ', '族': '족','現': '현 ', '比': '비 ', '發':'발 ', '兆': '조',
    #                       '檢': '검찰','警': '경찰','軍':'군','靑':'청와대 ', '法': '법원', '銀':'은행',
    #                       '上':'상','下':'하', 
    #                       '金': '김', '李':'이', '朴':'박', '文':'문 '
    #                       })
    #%:,./();#+ 
    #input=re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9%:,./();#+ ]", ' ',input)
    #input=re.sub(" +", ' ',input)
    #input=re.sub(r'[\.]+(?=[\.])', '',input)
    #input=input.replace('._//_', '.')
    #input=input.replace('_//_', '.')
    input = html.unescape(input)

    return str(input)

print(multi_re(q))

2020년 한국 TV 2대중 1대 인터넷 연결된다._//_ 아미코젠, 52주 신고가_//_ 3 


In [ ]:
print(raw_test['content'].head())
print(raw_test['title'].head())


0     마이크로 LED TV 장비 양산 돌입- 전방업체 투자 확대로 본업도 호조연일 '신고가'
1    [이데일리 김대웅 기자] 반도체 장비 업체 코세스(089890)의 기술력이 마이크로...
2    최근 대형 업체들과 거래를 맺고 관련 장비들의 양산에 돌입하면서 주가도 연일 신고가...
3    TV를 필두로 올해부터 마이크로 LED의 시대가 본격적으로 개화할 것으로 예상되면서...
4    코세스는 반도체 장비를 제조, 판매하는 업체로 지난 2006년 11월 코스닥 시장에...
Name: content, dtype: string
0    [주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대
1    [주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대
2    [주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대
3    [주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대
4    [주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대
Name: title, dtype: string


In [ ]:
#아노말리 제거
anomally=raw_train.copy()
anomally['content']=anomally['content']+anomally['title']
anomally=anomally.drop(['title', 'date'], axis=1)

anomally=anomally[anomally.duplicated(subset='content', keep=False)]
anomally['content']=anomally['content']+'_'+anomally['info']
anomally=anomally[~anomally.duplicated(subset='content', keep=False)].sort_values(by='content')

anomally['id']=anomally['n_id']+'_'+anomally['ord']
anomally=anomally[['id']]

In [ ]:
###train data 전처리###

#아노말리 제거
raw_train['id']=raw_train['n_id']+'_'+raw_train['ord']
raw_train=raw_train[~raw_train['id'].isin(anomally['id'])]
print('anomally: ', raw_train.shape)

#셔플
raw_train=raw_train.sample(frac=1, random_state=random_seed).reset_index(drop=True)

#제목과 컨텐츠 
raw_train['content']=raw_train['content']+'. '+raw_train['title']+'. '+raw_train['ord']
raw_test['content']=raw_test['content']+'. '+raw_test['title']+'. '+raw_test['ord']


In [ ]:
#한글, 영문, 숫자, 구두점 외 특수문자 공백으로 대체
raw_train['content']=raw_train.apply(lambda row : multi_re(row['content']), axis = 1) 

# 여러 컬럼 중 content, info만 추출
full_train=raw_train[['content', "info"]]
print(full_train.shape)
#중복 제거
full_train.drop_duplicates(subset=['content'], inplace=True)
print(full_train.shape)

(118745, 2)
(118334, 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
raw_train.content

0         무순위 청약이다보니 만 19세 이상의 수도권 거주자라면 누구나 가능하다. "3년 전...
1            응찰액 : 0.00조원. 한은 7차 이어 8차 시장안정용 무제한 RP매입 없음. 3
2         "주식카톡방" 대박주를 추천받는데 100원도 안낸다~. ‘하남시 시민안전보험에 관한...
3         기간 무제한. 이용료 0원, 주식카톡방 1천 3백명. 닫히기직전, 지금 바로 입장....
4         25일 펀드 스퀘어에 따르면, NH아문디자산운용의 NH-Amundi필승코리아증권투자...
                                ...                        
118740    이베스트투자증권은 CJ CGV에 대해 신종 코로나바이러스 감염증(코로나19) 영향으...
118741    참조가격 : 호가제출직전의 단일가격으로 시가 결정 전에는 당일 기준가격이, 시가 결...
118742    지난 2017년 시작된 사업은 시 청사 및 공공기간과 학교, 민간분야가 활용하는 다...
118743    종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지. 수도권 교육감 3명, ...
118744    10%수익을 40%로 만들어 준다고.. 영웅스탁론이라면 가능~.효과적인 레버리지를 ...
Name: content, Length: 118745, dtype: object

In [ ]:
print(raw_train.shape)

(118745, 6)


In [ ]:
#train 데이터셋
row, _=full_train.shape
train=full_train    #.iloc[0:int(row*0.8)]
#valid=full_train.iloc[int(row*0.8):]
test=raw_test[['id', 'content']]

print('train shape: ', train.shape)
#print('valid shape: ', valid.shape)
print('test shape: ', test.shape)

train shape:  (118334, 2)
test shape:  (142565, 2)


In [ ]:
###test 데이터셋 전처리###

#특수문자 제거
test['content']=test.apply(lambda row : multi_re(row['content']), axis = 1) 

#임시로 레이블 고정
test['info']='0'
#content, info 추출
test=test[['content', 'info']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
#데이터셋을 array로 변경 및 마지막 데이터셋 확인
dataset_train=train.values
#dataset_valid=valid.values
dataset_test=test.values

print('train dataset: ', dataset_train[-1])
#print('valid dataset: ', dataset_valid[-1])
print('test dataset: ', dataset_test[-1])

train dataset:  ['10%수익을 40%로 만들어 준다고.. 영웅스탁론이라면 가능~.효과적인 레버리지를 이용하여 수익률을 4배로... [김영상의 오지랖] “정치풍자, 섹드립 훌훌 다 벗고…” 한때 팬으로서의 자니윤 추모. 381'
 '1']
test dataset:  ['2020년 한국 TV 2대중 1대 인터넷 연결된다. 아미코젠, 52주 신고가. 76' '0']


In [ ]:
#데이터 분포 확인???????????????
#실험실
print('over_0: ', len([x for x in raw_train.values if x[2]==0]))
print('over_1: ', len([x for x in raw_train.values if x[2]==1]))

t=[[i, v]for i , v in enumerate(dataset_train) if str(v[1]).isdigit()==False]
print(t)
print('maxlen: ', max([(len(x[0]), idx) for idx, x in enumerate(train.values)]))
raw_train=raw_train[['date', 'ord', 'info']].astype('int')
print(raw_train.corr(method='pearson'))

over_0:  0
over_1:  0
[]
maxlen:  (3494, 73967)
          date       ord      info
date  1.000000  0.012890  0.028591
ord   0.012890  1.000000  0.124056
info  0.028591  0.124056  1.000000


In [ ]:
#토크나이저 로드
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
#학습에 사용될 데이터셋으로 변경하는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
        

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
#학습 파라미터 설정
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#데이터셋 변경 & 확인
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
#data_valid = BERTDataset(dataset_valid, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

print('train: ', data_train.sentences[-1])
#print('valid: ', data_valid.sentences[-1])
print('test: ', data_test.sentences[-1])

train:  (array([   2,  533,  517,   11, 2899,  606,  517,   11,  517, 6079, 1943,
       4248, 5439,  517,   54,  517,   54, 3376, 7017, 6664, 7589, 6084,
       7096, 6009,  737,  517,  463,  517,   54, 5173, 7119, 1884, 6324,
       7329, 3726, 7815, 2897, 7088,  605, 6312, 6079,  517,   54,  517,
         54,  517,   54,  702, 1326, 6527, 7095, 3417, 7318,    0,  517,
        363,  714, 4122, 7750, 7147,  517,   46,  517,    3], dtype=int32), array(64, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32))
test:  (array([   2,  554,  127, 5712, 4958,  694,  553, 5808, 7295,  529, 5808,
       3794, 3334, 5900,  517,   54, 3093, 6255, 7533, 7241,  517,   46,
        517,  180, 7276, 3012, 5330,  517,   54,  517,  215,    3,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1

In [ ]:
#torch dataloader로 변경
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
#valid_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
#kobert 분류기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)

        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
       
        if self.dr_rate:
            out = self.dropout(pooler)

        return self.classifier(out)

In [ ]:
#모델 선언
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

In [ ]:
#accuracy 계산 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
#이후 모델 파일명 및 submission에 사용하기 위한 변수 선언
answer=[]
acc=0
#starttime=0
#학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    #valid_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #model.eval()
    #for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(valid_dataloader)):
    #    token_ids = token_ids.long().to(device)
    #    segment_ids = segment_ids.long().to(device)
    #    valid_length= valid_length
    #    label = label.long().to(device)
    #    out = model(token_ids, valid_length, segment_ids)
    #    valid_acc += calc_accuracy(out, label)
    #print("epoch {} valid acc {}".format(e+1, valid_acc / (batch_id+1)))

    if e==num_epochs-1:      
        with torch.no_grad():
            #학습이 끝나면 submission할 결과 생성
            #starttime=time.time()
            print('training ended. making submission file')
            #acc=valid_acc / (batch_id+1)
            acc='0.0000'
            model.eval()
            for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
                token_ids = token_ids.long().to(device)
                segment_ids = segment_ids.long().to(device)
                valid_length= valid_length
                label = label.long().to(device)
                out = model(token_ids, valid_length, segment_ids)
                out=out.cpu()
                tmp=[torch.argmax(i) for i in out]
                answer=answer+tmp

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


epoch 1 batch id 1 loss 0.7714811563491821 train acc 0.421875
epoch 1 batch id 201 loss 0.03220372274518013 train acc 0.8880597014925373
epoch 1 batch id 401 loss 0.03941165655851364 train acc 0.9381429239401496
epoch 1 batch id 601 loss 0.003201201558113098 train acc 0.956101809484193
epoch 1 batch id 801 loss 0.017920460551977158 train acc 0.9655898876404494

epoch 1 train acc 0.9696362794245859


epoch 2 batch id 1 loss 0.001751502975821495 train acc 1.0
epoch 2 batch id 201 loss 0.0009749671444296837 train acc 0.9970071517412935
epoch 2 batch id 401 loss 0.0002194531261920929 train acc 0.9976036471321695
epoch 2 batch id 601 loss 0.0008754367008805275 train acc 0.9979331322795341
epoch 2 batch id 801 loss 0.0008714944124221802 train acc 0.998185861423221

epoch 2 train acc 0.9983699324324324
training ended. making submission file


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
#valid acc에 따라 모델 이름 변경 & 저장
import os
acc=str(acc).replace('.', "")[1:5]

folder_path = '/gdrive/My Drive/models/kobert_{}_{}'.format(acc, status)
file_path=folder_path+'/model.pth'
if not os.path.isdir(folder_path): os.mkdir(folder_path)

torch.save(model.state_dict(), file_path)

In [ ]:
#test id와 분류 결과 병합
info=pd.Series(np.array(answer), name='info')
id=raw_test["id"]
submission=pd.DataFrame(pd.concat([id, info], axis=1))

print(submission.head())
print('submission shape: ', submission.shape)

            id  info
0  NEWS00237_1     0
1  NEWS00237_2     0
2  NEWS00237_3     0
3  NEWS00237_4     0
4  NEWS00237_5     0
submission shape:  (142565, 2)


In [ ]:
#제출 파일 저장
submission.to_csv(path.join(folder_path, 'submission.csv'), encoding='utf-8', header=['id', 'info'], index=None, sep=',')
#코드 백업
#copy(path.join('/gdrive/My Drive/Colab Notebooks/', 'naver_review_classifications_pytorch_kobert.ipynb'), folder_path)